In [ ]:
import pyalex
from pyalex import Works, Authors, Sources, Institutions, Topics, Publishers, Funders

pyalex.config.email = "thomas.hoger@laas.fr"

In [61]:
import json 

keywords = json.load(open("key_words_classic.json"))

keywords += ["pfcp", "gtp", "http api", "ngap", "nas", "3GPP standard", "MITRE", "FIGHT"]

In [67]:
import tqdm

refs = {}
key_to_keep = ['title', 'doi', 'relevance_score', 'publication_year', 'cited_by_count']

for keyword in tqdm.tqdm(keywords) :
    
    research_texts = [
        f"5G security {keyword}",
        f"5G vulnerabilities {keyword}",
        f"5G attacks {keyword}"
    ]
    
    for research_text in research_texts:
    
        results = Works().search_filter(title=research_text).get()
        results = [{k: v for k, v in res.items() if k in key_to_keep} for res in results]
        refs[research_text] = results
    
json.dump(refs, open("refs_pre_process.json", "w"))

100%|██████████| 199/199 [02:19<00:00,  1.43it/s]


In [ ]:
banned_keywords = [
    "ue",
    "roaming",
    "supply chain",
    "ran",
    "controller",
    "virtualization",
    "radio interface",
    "vswitch",
    "password",
    "tool",
    "mec",
    "network access"
]

processed = {}
for keyword, ref_list in refs.items(): 
    
    found = False
    for banned_keyword in banned_keywords:
        if banned_keyword in keyword :
            found = True
            break
    
    if not found and len(ref_list) > 0:
        processed[keyword] = ref_list

json.dump(processed, open("refs_post_process.json", "w"))


In [85]:
import pandas as pd

all_ref = []
for keyword, ref_list in processed.items(): 
    all_ref += ref_list

df = pd.DataFrame(all_ref) 
print(f"Number of keywoards : {len(keywords)}")
print(f"Number of ref : {len(df)}")
print("\nDropping dupplicates...")

df = df.drop_duplicates(subset=["doi"])
print(f"Number of ref : {len(df)}")

# df = df[df['cited_by_count'] > 2]
# print(f"With 1+ citation : {len(df)}")

Number of keywoards : 199
Number of ref : 117

Dropping dupplicates...
Number of ref : 104


In [86]:
df_sorted = df.sort_values(by='relevance_score', ascending=False)
df_sorted.to_csv("refs_filtered.csv", index=False)

In [5]:
import pandas as pd

df = pd.read_csv("refs_filtered.csv")
df = df[df['title'].str.contains('slic', case=False, na=False)]
len(df)

52